In [29]:
import pandas as pd
import numpy as np
import os
import json
# import dt
import re

# GM_RESULT

In [30]:
res = pd.read_csv("game_results.csv")
res.head()


,game_id,seed_game_id,players,generations,game_options,scores,createtime
0,f7cbeb6cc687,NaN,4,8,NaN,"[{""corporation"":""Poseidon"",""playerScore"":69,""p...",31/8/2020 15:05:21
1,10d3f58364b6,NaN,2,12,NaN,"[{""corporation"":""Aridor"",""playerScore"":144,""pl...",31/8/2020 15:05:21
2,ff2c9f3f3e45,NaN,4,8,NaN,"[{""corporation"":""Inventrix (breakthrough)"",""pl...",31/8/2020 15:05:21
3,cf45c3f6faa2,NaN,2,11,NaN,"[{""corporation"":""Arklight (breakthrough)"",""pla...",31/8/2020 15:05:21
4,79cef4b6a297,NaN,2,8,NaN,"[{""corporation"":""Point Luna"",""playerScore"":60,...",31/8/2020 15:05:21


In [31]:
# preprocessing
res["scores"] = res["scores"].astype(object)
res["game_options"] = res["game_options"].astype(object)
res["createtime"] = pd.to_datetime(res["createtime"], dayfirst=True)


In [32]:
# scores proprocessing
store = []
for i in res["scores"]:
    tmp = json.loads(i)
    for i in range(len(tmp)):
        tmp[i]['order'] = i+1
    tmp = sorted(tmp,key = lambda e:e.__getitem__('playerScore'),reverse = True) # can optimize to obtain knowledge of order
    for i in range(len(tmp)):
        tmp[i]['position'] = i+1
    store.append(tmp)
df1 = pd.DataFrame(store)
df1.columns = ["player1","player2","player3","player4","player5","player6"]

In [33]:
options = []
for i in res["game_options"]:
    if not pd.isna(i):
        options.append(json.loads(i))
    else: options.append({})
df2 = pd.DataFrame(options)

In [34]:
df2.head()

,draftVariant,corporateEra,preludeExtension,venusNextExtension,coloniesExtension,turmoilExtension,boardName,showOtherPlayersVP,customCorporationsList,customColoniesList,...,showTimers,politicalAgendasExtension,moonExpansion,requiresMoonTrackCompletion,erosCardsOption,politicalAgendasExtensionToggle,showCardsBlackList,showColoniesList,randomMAToggle,doubleCorp
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df3 = pd.concat([res, df1, df2],axis=1)
df4= df3.drop(columns=['seed_game_id','game_options','scores','draftVariant','corporateEra','solarPhaseOption','undoOption','fastModeOption','includeVenusMA','soloTR','removeNegativeGlobalEventsOption','politicalAgendasExtension','requiresMoonTrackCompletion','showTimers','politicalAgendasExtensionToggle','aresHazards','requiresVenusTrackCompletion','politicalAgendasExtensionToggle','solarPhaseOption'])

In [36]:
df4.columns
df4.to_csv('./prprocessing_data', index = False)

In [37]:
def getPlayer(df,num): # get all players name
    player_df = pd.DataFrame(columns=['player','count'])
    for index,i in df.iterrows():
        if i['players'] == num: #start
            for column in ['player1','player2','player3','player4']:
                player_df.loc[player_df.shape[0]] = [i[column]['player'].lower().strip(),1]
    ff1 = player_df.groupby(player_df["player"]).count()
    ff1["player_list"] = ff1.values.tolist()
    return ff1


In [38]:
player_count = getPlayer(df4,4)
player_count.to_csv('./pl_count.csv', index = True)

In [39]:
def playerCount(df,num,player): #TODO,模糊查找
    player_df = pd.DataFrame(columns=['id','corporation1', 'rank', 'order','score', 'time', 'generations','corporation2'])
    for index,i in df.iterrows():
        if i['players'] == num: #start
            for column in ['player1','player2','player3','player4']:
                if i[column]['player'].lower() in [p.lower() for p in player] and ('corporation2' not in i[column].keys()):
                    player_df.loc[player_df.shape[0]] = [i[column]['player'].lower(),i[column]['corporation'],i[column]['position'],i[column]['order'],i[column]['playerScore'],i['createtime'],i['generations'],'nan']
                elif i[column]['player'].lower() in [p.lower() for p in player] and ('corporation2' in i[column].keys()):
                    player_df.loc[player_df.shape[0]] = [i[column]['player'].lower(),i[column]['corporation'],i[column]['position'],i[column]['order'],i[column]['playerScore'],i['createtime'],i['generations'],i[column]['corporation2']]
    return player_df

In [40]:
df_player = playerCount(df4,4,['ee'])
df_player.tail(20)

,id,corporation1,rank,order,score,time,generations,corporation2
0,ee,Aphrodite (breakthrough),2,1,82,2021-04-26 16:26:26,7,nan
1,ee,Manutech,1,1,74,2021-04-27 17:34:43,7,nan
2,ee,Inventrix (breakthrough),4,2,76,2021-04-28 01:41:52,8,nan
3,ee,Terralabs Research (breakthrough),1,1,85,2021-04-28 17:40:15,8,nan
4,ee,Philares,1,1,65,2021-04-29 01:01:53,5,nan
5,ee,Teractor (breakthrough),1,3,76,2021-04-30 01:36:51,7,nan
6,ee,Pharmacy Union,1,3,102,2021-05-20 00:42:47,7,nan
7,ee,Lakefront Resorts,2,2,66,2021-06-20 00:12:36,6,Mining guild (breakthrough)
8,ee,Philares,1,4,80,2021-07-03 14:13:03,6,Ecoline (breakthrough)
9,ee,Point Luna,1,3,87,2021-07-09 20:50:31,6,Teractor (breakthrough)


In [41]:
#time series
def time_series(df):
    ff1 = df[(df["players"]==4) & (df["generations"]<=9)]
    ff1.to_csv('./time_original', index = True)
    ff2 = ff1.groupby(ff1['createtime'].dt.date).mean()
    return ff2


In [42]:
# 时间绘图
df5 = time_series(df4)
df5.head(20)
df5 = df5.drop(columns=["players","initialDraftRounds"])
df5.to_csv('./time_series.csv', index = True)
# df5.plot()

# US_RESULT

In [43]:
# usage: demostrate user's creating time, like "you are the NO.1132 player"
user = pd.read_csv("users.csv")
user = user.drop(columns="prop")
user["name"] = user["name"].str.lower()
user["createtime"] = pd.to_datetime(user["createtime"], dayfirst=True)
user = user.sort_values(by=["createtime"])
user = user.reset_index(drop=True)
user["createtime"] = user.index+1
user.to_csv('./user_pre.csv', index = False)

In [44]:
# left join and mapping
user_player = user.merge(player_count,how="left",left_on="name",right_on="player")
user_player = user_player.sort_values(by=["player_list"],na_position="first")
user_player.to_csv('./user_player_mapping.csv', index = False)
user_player = user_player.set_index("name")
user_player = user_player.drop(columns="player_list")
user_player.tail()

,createtime,count
name,,
奶牛,83,154.0
阿尔,294,159.0
无语,21,176.0
钝钝,59,269.0
乙木,17,301.0


In [45]:
# detect missing user name
player_user = player_count.merge(user,how="left",left_on="player",right_on="name")
player_user = player_user.sort_values(by=["count"],ascending=False, na_position="first")
player_user = player_user[player_user["name"].isna()]
player_user.to_csv('./player_user_mapping.csv', index = False)
player_user.head()

,count,player_list,name,createtime
1443,238,[238],NaN,NaN
1302,210,[210],NaN,NaN
1444,121,[121],NaN,NaN
222,120,[120],NaN,NaN
349,106,[106],NaN,NaN


# CUSTOMIZATION

In [46]:
player_mapping_df = pd.read_csv('player_mapping.csv')
player_mapping_df = player_mapping_df.set_index("playername")

In [47]:
def playerGroup(df,num,player,map_df): #TODO,模糊查找
    player_df = pd.DataFrame(columns=['id','corporation1', 'rank', 'order','score', 'time', 'generations','corporation2'])
    for index,i in df.iterrows():
        if i['players'] == num: #start
            for column in ['player1','player2','player3','player4']:
                if (i[column]['player'].lower() == player.lower() or i[column]['player'].lower() in map_df.index) and ('corporation2' not in i[column].keys()):
                    if map_df.loc[i[column]['player'].lower(),'username']== player.lower():
                        player_df.loc[player_df.shape[0]] = [i[column]['player'].lower(),i[column]['corporation'],i[column]['position'],i[column]['order'],i[column]['playerScore'],i['createtime'],i['generations'],'nan']
                elif (i[column]['player'].lower() == player.lower() or i[column]['player'].lower() in map_df.index) and ('corporation2' in i[column].keys()):
                    if map_df.loc[i[column]['player'].lower(),'username']== player.lower():
                        player_df.loc[player_df.shape[0]] = [i[column]['player'].lower(),i[column]['corporation'],i[column]['position'],i[column]['order'],i[column]['playerScore'],i['createtime'],i['generations'],i[column]['corporation2']]
    return player_df

In [153]:
user_df = playerGroup(df4,4,'小黄',player_mapping_df)
user_df.head()

,id,corporation1,rank,order,score,time,generations,corporation2


In [152]:
group_by_corp = user_df.groupby(['corporation1']).agg({'id':'count','rank':'mean','score':'mean','generations':'mean'}).sort_values(by=['id'], ascending=False)

user_df["hour"] = user_df.time.apply(lambda x: x.hour)
group_by_hour = user_df.groupby(['hour']).agg({'id':'count','rank':'mean','score':'mean','generations':'mean'}).sort_values(by=['id'], ascending=False)
group_by_hour.reset_index().loc[0,'hour']
highest = user_df.sort_values(by=["score","generations"],ascending=[False,True])
highest.head()


KeyError: 0

In [50]:
user_df.to_csv('ee.csv',index=False)

In [94]:
# test
firstgame = user_df.loc[0,["corporation1","rank","time"]]
user_df["date"] = user_df["time"].dt.normalize()
group_by_date = user_df.groupby(['date']).agg({'id':'count'}).sort_values(by=['id'], ascending=False)
group_by_date = group_by_date[group_by_date["id"]<=7]
group_by_date.reset_index().head()

,date,id
0,2021-04-13,2
1,2021-04-22,2
2,2020-12-11,2
3,2020-12-12,2
4,2021-04-15,2


In [177]:
def user_summary(ff1,ff2,user): ## ff1=df4
    user_df = playerGroup(ff1,4,user,player_mapping_df)
    # measure 0: check whether the account is uesable
    if user_df.empty: return [''for i in range(18)]
    if pd.isna(ff2.loc[user,"createtime"].astype("int")): return [''for i in range(18)]
    # measure 1: account create date
    createtime = ff2.loc[user,"createtime"]
    # measure 1.5: first game
    firstgame = user_df.loc[0,["corporation1","rank","time"]]
    # measure 1.6 spend how many days, defeat ?% users
    # measure 2: total games, favourite corp
    group_by_corp = user_df.groupby(['corporation1']).agg({'id':'count','rank':'mean','score':'mean','generations':'mean'}).sort_values(by=['id'], ascending=False)
    group_by_corp = group_by_corp.reset_index()
    favorite_corp = group_by_corp.loc[0,'corporation1']
    favourite_corp_rank = group_by_corp.loc[0,'rank']
    favorite_corp_times = group_by_corp.loc[0,'id']
    games = user_df.shape[0]
    # measure 3: most frequently play time period
    user_df["hour"] = user_df.time.apply(lambda x: x.hour)
    group_by_hour = user_df.groupby(['hour']).agg({'id':'count','rank':'mean','score':'mean','generations':'mean'}).sort_values(by=['id'], ascending=False)
    most_freq_time = group_by_hour.reset_index().loc[0,'hour']
    # measure 4: avg score and gen, i.e. defeat 96% players (display this if the ratio > 50) TODO
    avg_rank = user_df["rank"].mean()
    avg_gens = user_df["generations"].mean()
    # measure 5: highest score, with which corp
    # highest = pd.to_numeric(user_df["score"]).idxmax()
    highest = user_df.sort_values(by=["score","generations"],ascending=[False,True])
    highest_corp = highest.iloc[0,1]
    highest_score = highest.iloc[0,4]
    highest_gens = highest.iloc[0,6]
    highest_corp = highest_corp+' and '+user_df.iloc[0,7] if user_df.iloc[0,7] != 'nan' else highest_corp
    # measure 6: group by day. i.e 2021-06-10 seems to be a special day for u, u played 6 games by a row
    user_df["date"] = user_df["time"].dt.normalize()
    group_by_date = user_df.groupby(['date']).agg({'id':'count'}).sort_values(by=['id'], ascending=False)
    group_by_date = group_by_date[group_by_date["id"]<=7]
    played_day = group_by_date.shape[0]
    group_by_date = group_by_date.reset_index()
    most_day = group_by_date.loc[0,["date","id"]]
    # print([user, firstgame[0],firstgame[1],firstgame[2],createtime, games,favorite_corp,favorite_corp_times,most_freq_time,avg_rank,avg_gens, highest_corp, highest_score,highest_gens,played_day,most_day[0], most_day[1]])
    return [user, firstgame[0],firstgame[1],firstgame[2],createtime, games,favorite_corp,favorite_corp_times,favourite_corp_rank,most_freq_time,avg_rank,avg_gens, highest_corp, highest_score,highest_gens,played_day,most_day[0], most_day[1]]

In [178]:
def measure_summary(ff1,ff2):
    output_df = pd.DataFrame(columns=["user_id","first_corp","first_rank","first_time","create_no","total_games","favorite_corp","fa_corp_times","fa_corp_rank","most_freq_play_time","avg_rank","avg_gens","highest_corp","highest_score","highest_gens","played_day","most_date","most_date_times"])
    for index,i in user_player.iterrows():
        # print(index)
        if (not pd.isnull(i["count"])) and i["count"]>1:
            output_df.loc[output_df.shape[0]] = user_summary(ff1,ff2,index)
    return output_df

In [179]:
print(user_summary(df4,user_player,'小黄'))

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [180]:
output_df = measure_summary(df4,user_player)
output_df = output_df.dropna()
output_df.to_csv("measure_summary.csv")
output_df.head()


,user_id,first_corp,first_rank,first_time,create_no,total_games,favorite_corp,fa_corp_times,fa_corp_rank,most_freq_play_time,avg_rank,avg_gens,highest_corp,highest_score,highest_gens,played_day,most_date,most_date_times
0,boxness,Tharsis Republic,3,2021-06-20 23:15:05,404.0,2,Teractor,1,2.0,22,2.5,9.0,Teractor and,94,10,2,2021-06-20 00:00:00,1
1,deng,Teractor,3,2020-12-05 23:59:27,398.0,2,Arcadian Communities,1,2.0,23,2.5,10.5,Teractor,102,12,1,2020-12-05 00:00:00,2
2,blake,Teractor (breakthrough),4,2020-12-09 20:19:41,84.0,2,Recyclon (breakthrough),1,4.0,13,4.0,7.5,Recyclon (breakthrough),50,7,2,2020-12-09 00:00:00,1
3,tm-tour-101,Vitor,3,2021-02-02 21:27:23,148.0,2,Utopia Invest,1,2.0,20,2.5,8.0,Vitor,69,9,2,2021-02-02 00:00:00,1
4,laola,Valley Trust,4,2021-05-01 20:52:13,361.0,2,Valley Trust,1,4.0,20,2.5,8.0,Viron,86,9,2,2021-05-01 00:00:00,1


In [ ]:
def bayesian_avarage(rank,times):
    return (rank*times+2.5*10)/times+10

In [215]:
output_df = pd.read_csv("measure_summary.csv")
# preprocessing
output_df = output_df[output_df["user_id"].str.contains("tour-")==False]
# play times
output_df = output_df.sort_values(by="total_games",ascending=False)
output_df['play_times_sort'] = range(1,len(output_df)+1)
# gens
output_df = output_df.sort_values(by="avg_gens",ascending=False)
output_df['gens_sort'] = range(1,len(output_df)+1)
# terraform
output_df = output_df.sort_values(by="avg_gens")
output_df['terra_sort'] = range(1,len(output_df)+1)
# rank
output_df["bayes_rank"] = output_df.apply(lambda x:(x["total_games"]*x["avg_rank"]+1i*2.8)/(x["total_games"]+10),axis=1)
output_df = output_df.sort_values(by="bayes_rank")
output_df['rank_sort'] = range(1,len(output_df)+1)
# analyze corp ranking TODO: group by corp,ignore 'breakthrough', then find the index with the maximum values
output_df['baye_avg'] = output_df.apply(lambda x:(x["fa_corp_times"]*x["fa_corp_rank"]+5*2.5)/(x["fa_corp_times"]+5),axis=1)
output_df['max_corp'] = output_df.groupby(["favorite_corp"],as_index=False)["baye_avg"].transform(min)
output_df['max_corp'] = np.where((output_df["max_corp"]==output_df["baye_avg"]) & (output_df["baye_avg"] <= 2.5),1,0)
# output_pivot = pd.pivot_table(output_df, index=[''])
# output_df.sort_values(by="baye_avg").groupby(["favorite_corp"],as_index=False).transform(lambda x: x in range(1,len(output_df)+1))


output_df = output_df.sort_values(by="max_corp",ascending=False)
output_df.to_csv("measure_summary_sort.csv")
output_df.head()

,Unnamed: 0,user_id,first_corp,first_rank,first_time,create_no,total_games,favorite_corp,fa_corp_times,fa_corp_rank,...,played_day,most_date,most_date_times,play_times_sort,gens_sort,terra_sort,bayes_rank,rank_sort,baye_avg,max_corp
287,287,ender,Poseidon,4.0,2020-08-31 15:05:21,2.0,157.0,Manutech,11.0,1.636364,...,92.0,2021-04-13 00:00:00,2.0,11,189,21,1.832335,1,1.906250,1
226,226,827717358,Saturn Systems,3.0,2021-05-31 16:14:28,862.0,14.0,Septem Tribus,2.0,1.500000,...,11.0,2021-06-17 00:00:00,3.0,71,197,13,2.875000,165,2.214286,1
129,129,封云台,Thorgate,4.0,2021-04-30 22:14:09,718.0,3.0,PhoboLog,1.0,1.000000,...,3.0,2021-04-30 00:00:00,1.0,147,5,205,2.615385,84,2.250000,1
236,236,jequirity,EcoLine,2.0,2020-12-19 17:57:12,287.0,18.0,Tharsis Republic,3.0,1.666667,...,15.0,2021-02-26 00:00:00,3.0,65,99,107,2.607143,81,2.187500,1
39,39,维尼熊,Factorum (breakthrough),4.0,2021-07-22 17:14:23,1211.0,2.0,Curiosity II,1.0,2.000000,...,2.0,2021-07-22 00:00:00,1.0,186,142,74,2.833333,159,2.416667,1


In [226]:
# final json file
final_df = pd.read_csv("measure_summary_sort.csv")
# entity modification
final_df["create_no"] = final_df["create_no"].astype(np.int32)
final_df[["create_time_text","first_game_text","total_game_text","favorite_corp_text","favorite_time_text","avg_rank_text","avg_gen_text","best_score_text","games_in_one_day_text","achievement_text"]] = final_df.apply(lambda x:('','','','','','','','','','',),axis=1,result_type='expand')
# for loop
for i,row in final_df.iterrows():
    # 1
    if final_df.loc[i,"create_no"] <= 100: final_df.loc[i,"create_time_text"] = "作为登陆火星的先行者，你们为火星的建设奠定了坚实的基础！"
    elif final_df.loc[i,"create_no"] <= 805: final_df.loc[i,"create_time_text"] = "作为火星的第二批居民，你们牵引了无数小行星让火星不再孤寂寒冷。"
    else: final_df.loc[i,"create_time_text"] = "作为火星的新兴势力，感谢你们为火星增添了几分绿意！"

    # 2
    if final_df.loc[i,"create_no"] <= 100: final_df.loc[i,"create_time_text"] = "作为登陆火星的先行者，你们为火星的建设奠定了坚实的基础！"
    elif final_df.loc[i,"create_no"] <= 805: final_df.loc[i,"create_time_text"] = "作为火星的第二批居民，你们牵引了无数小行星让火星不再孤寂寒冷。"
    else: final_df.loc[i,"create_time_text"] = "作为火星的新兴势力，感谢你们为火星增添了几分绿意！"
final_df.to_csv("measure_output.csv")
final_df.head()

,Unnamed: 0,Unnamed: 0.1,user_id,first_corp,first_rank,first_time,create_no,total_games,favorite_corp,fa_corp_times,...,create_time_text,first_game_text,total_game_text,favorite_corp_text,favorite_time_text,avg_rank_text,avg_gen_text,best_score_text,games_in_one_day_text,achievement_text
0,287,287,ender,Poseidon,4.0,2020-08-31 15:05:21,2,157.0,Manutech,11.0,...,作为登陆火星的先行者，你们为火星的建设奠定了坚实的基础！,,,,,,,,,
1,226,226,827717358,Saturn Systems,3.0,2021-05-31 16:14:28,862,14.0,Septem Tribus,2.0,...,作为火星的新兴势力，感谢你们为火星增添了几分绿意！,,,,,,,,,
2,129,129,封云台,Thorgate,4.0,2021-04-30 22:14:09,718,3.0,PhoboLog,1.0,...,作为火星的第二批居民，你们牵引了无数小行星让火星不再孤寂寒冷。,,,,,,,,,
3,236,236,jequirity,EcoLine,2.0,2020-12-19 17:57:12,287,18.0,Tharsis Republic,3.0,...,作为火星的第二批居民，你们牵引了无数小行星让火星不再孤寂寒冷。,,,,,,,,,
4,39,39,维尼熊,Factorum (breakthrough),4.0,2021-07-22 17:14:23,1211,2.0,Curiosity II,1.0,...,作为火星的新兴势力，感谢你们为火星增添了几分绿意！,,,,,,,,,
